In [ ]:
# Install required packages
!pip install transformers torch pandas scikit-learn matplotlib datasets tqdm

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch

# Load the dataset
file_path = '/content/data/2024_4.csv'
news_data = pd.read_csv(file_path)

# Display the first few rows of the dataset
news_data.head()

# Explore the dataset
print(news_data['Category'].value_counts())

# Filter necessary columns and drop any rows with NaN values
news_data = news_data[['Title', 'Category']].dropna().reset_index(drop=True)

# Encode labels
labels, uniques = pd.factorize(news_data['Category'])
news_data['label'] = labels

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(news_data['Title'], news_data['label'], test_size=0.2, random_state=42)

# Ensure the texts are in the correct format and reset indices
train_texts = train_texts.reset_index(drop=True).tolist()
test_texts = test_texts.reset_index(drop=True).tolist()
train_labels = train_labels.reset_index(drop=True).tolist()
test_labels = test_labels.reset_index(drop=True).tolist()

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# Create Dataset objects
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
test_dataset = NewsDataset(test_encodings, test_labels)

# Load the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(uniques))

# Move model to the correct device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="steps",     # evaluate every `logging_steps`
    eval_steps=100,                  # evaluate every 100 steps
    save_steps=100,                  # save the model every 100 steps
    load_best_model_at_end=True,     # load the best model at the end of training
)

# Create a Trainer instance
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

# Train the model
trainer.train()

# Evaluate the model
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

print('Accuracy:', accuracy_score(test_labels, predictions))
print('Classification Report:', classification_report(test_labels, predictions, target_names=uniques))

# Function to classify new headlines
def classify_headline(headline):
    inputs = tokenizer(headline, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the correct device
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return uniques[predicted_class]

# Example usage
print(classify_headline("New breakthrough in AI technology"))

# Plot training and evaluation loss
history = trainer.state.log_history

train_loss = [log['loss'] for log in history if 'loss' in log]
eval_loss = [log['eval_loss'] for log in history if 'eval_loss' in log]
steps = [log['step'] for log in history if 'loss' in log]

plt.figure(figsize=(10, 5))
plt.plot(steps, train_loss, label='Training Loss')
plt.plot(steps, eval_loss, label='Evaluation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Evaluation Loss')
plt.legend()
plt.savefig('./results/training_logs.png')
plt.show()
